# Advent of Code 2021 - Day 16

In [53]:
EXAMPLE = 0
filename = "input.txt"
if EXAMPLE: filename = "example.txt"
    
data = []
with open(filename, "r") as f:
    for line in f:
        data += [line.strip()]
        
data[0]

'220D4B80491FE6FBDCDA61F23F1D9B763004A7C128012F9DA88CE27B000B30F4804D49CD515380352100763DC5E8EC000844338B10B667A1E60094B7BE8D600ACE774DF39DD364979F67A9AC0D1802B2A41401354F6BF1DC0627B15EC5CCC01694F5BABFC00964E93C95CF080263F0046741A740A76B704300824926693274BE7CC880267D00464852484A5F74520005D65A1EAD2334A700BA4EA41256E4BBBD8DC0999FC3A97286C20164B4FF14A93FD2947494E683E752E49B2737DF7C4080181973496509A5B9A8D37B7C300434016920D9EAEF16AEC0A4AB7DF5B1C01C933B9AAF19E1818027A00A80021F1FA0E43400043E174638572B984B066401D3E802735A4A9ECE371789685AB3E0E800725333EFFBB4B8D131A9F39ED413A1720058F339EE32052D48EC4E5EC3A6006CC2B4BE6FF3F40017A0E4D522226009CA676A7600980021F1921446700042A23C368B713CC015E007324A38DF30BB30533D001200F3E7AC33A00A4F73149558E7B98A4AACC402660803D1EA1045C1006E2CC668EC200F4568A5104802B7D004A53819327531FE607E118803B260F371D02CAEA3486050004EE3006A1E463858600F46D8531E08010987B1BE251002013445345C600B4F67617400D14F61867B39AA38018F8C05E430163C6004980126005B801CC0417080106005000CB4002D7A801AA0062

# Part 1

In [2]:
def add_leading_zeros_to_make_multiple_of_four(in_b):    
    if len(in_b) % 4 != 0:        
        new_len = len(in_b)
        while(True):
            new_len += 1
            if new_len % 4 == 0:
                break
        new = ('0' * (new_len - len(in_b))) + in_b
        return new
    
    else:
        return in_b
    
add_leading_zeros_to_make_multiple_of_four('111000000000000110111101000101001010010001001000000000')

'00111000000000000110111101000101001010010001001000000000'

In [3]:
class Packet_Node:
    def __init__(self):
        self.children = []
        self.version = None
        self.type_id = None
        self.num_bits = None
        self.begin = None
        self.literal = None
        self.end = None
        
    def Print_Tree(self, depth = 0):
        print('\t' * depth, 'Node (Size: {}):'.format(self.num_bits), ', Children ({})'.format(len(self.children)), ', Type ID:', self.type_id, ', Version:', self.version, ', Literal:', self.literal)
        for x in self.children:
            x.Print_Tree(depth + 1)
        
        
Packet_Node().Print_Tree()

 Node (Size: None): , Children (0) , Type ID: None , Version: None , Literal: None


In [4]:
def Grab_Literal(packet):
    
    literal_b = ''
    number = packet[6:]
    bit_count = 0
    for i in range(0, len(number), 5):
        chunk = number[i: i + 5]
        bit_count += 5
        if chunk[0] == '1':
            literal_b += chunk[1:]
        elif chunk[0] == '0':
            literal_b += chunk[1:]
            break
            
    remaining = packet[6 + bit_count: ]
    return literal_b, bit_count, remaining

In [56]:
def Create_Tree_From_Packets(packet, parent_node, num_children_packets = None, depth = 0):
    print('\nCreate_Tree_From_Packets:', packet, ', Read Limit:', num_children_packets, ', Recursion Depth:', depth)

    node = Packet_Node()
    parent_node.children += [node]

    version_b = packet[:3]
    type_id_b = packet[3:6]
    version = int(version_b, 2)
    type_id = int(type_id_b, 2)
    
    node.version = version
    node.type_id = type_id
    
    print('\tPacket:', version_b, type_id_b, packet[6:])
    print('\tVersion:', version_b, '=', version)
    if type_id == 4: print('\tType ID:', type_id_b, '=', type_id, '(Literal)')
    else: print('\tType ID:', type_id_b, '=', type_id)
        
    # Literal Packet
    if type_id == 4:
        print('\n\tLiteral Packet!\n')
        literal_b, bit_count, remaining = Grab_Literal(packet)
                
        print('\tLiteral (Size in bits: {}):'.format(len(literal_b)), literal_b, '=', int(literal_b, 2))
        print('\tEncoding Bit Count:', bit_count)
        
        print('\t{} {} {}'.format(version_b, type_id_b, packet[6:]))
        print('\t{} {} {}'.format('VVV', 'TTT', 'L' * bit_count))
        print('\tRemaining bits:', remaining)

        if len(remaining) and remaining != len(remaining) * '0':
            if num_children_packets:
                if num_children_packets > 1:
                    print('\tRecursing from literal with read limit on remaining!')
                    Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1)
            else:
                print('\tRecursing from literal on remaining!')
                Create_Tree_From_Packets(remaining, parent_node, depth = depth + 1)
        else:
            print('\tNo remaining or all zeros!')
            
        node.literal = int(literal_b, 2)
        node.num_bits = 3 + 3 + bit_count
        
    # Not Literal
    else:
        length_type_ID = packet[6]
        print('\n\tNot Literal!')
        print('\tLength Type ID:', length_type_ID)
        
        # Bit Count
        if length_type_ID == '0':
            print('\n\tBit Count!\n')
            total_length_b = packet[7 : 7  + 15]
            total_length = int(total_length_b, 2)
            node.num_bits = 7 + 15 + total_length

            print('\tTotal length in bits of sub-packets:', total_length_b, "=", total_length)

            children_packets = packet[7 + 15 : 7 + 15 + total_length]
            assert len(children_packets) == total_length
        
        
            print('\tChildren bits:', children_packets)
            
            print('\t{} {} {} {} {}'.format(version_b, type_id_b, length_type_ID, total_length_b, packet[7 + 15 :]))
            print('\t{} {} {} {} {}'.format('VVV', 'TTT', 'I', 'X' * 15, 'C' * len(children_packets)))
            
            print('\tRecursing from bit count on children!')
            Create_Tree_From_Packets(children_packets, node, depth = depth + 1)
            
            remaining = packet[7 + 15 + total_length: ]            
            print('\tRemaining bits:', remaining)

            if len(remaining) and remaining != len(remaining) * '0':
                if num_children_packets:
                    if num_children_packets > 1:
                        print('\tRecursing from bit count with read limit on remaining!')
                        Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1)
                else:
                    print('\tRecursing from bit count on remaining!', depth = depth + 1)
                    Create_Tree_From_Packets(remaining, parent_node)
            else:
                print('\tNo remaining or all zeros!')

        # Packet Count
        else:
            print('\n\tPacket Count!\n')
            num_packets_b = packet[7 : 7 + 11]
            num_packets = int(num_packets_b, 2)
            
            print('\tNumber of sub-packets immediately contained:', num_packets_b, "=", num_packets)
            
            all_after = packet[7 + 11 : ]
            print('\tAll after bits:', all_after)
            
            print('\t{} {} {} {} {}'.format(version_b, type_id_b, length_type_ID, num_packets_b, packet[7 + 11 :]))
            print('\t{} {} {} {} {}'.format('VVV', 'TTT', 'I', 'X' * 11, '?' * len(all_after)))
            

            print('\tRecursing from bit packet count on all after!')
            Create_Tree_From_Packets(all_after, node, num_packets, depth = depth + 1)
            
            print('\tExpected number of sub-packets:', num_packets, 'Actual Number of sub-packets', len(node.children))
            
            assert len(node.children) == num_packets 

            # At this point we know where it ends
            total_length = sum([x.num_bits for x in node.children])
            print('my total_length', [x.num_bits for x in node.children], total_length)
            node.num_bits = 7 + 11 + total_length
            
            remaining = packet[7 + 11 + total_length : ]            
            print('\tRemaining bits:', remaining)

            if len(remaining) and remaining != len(remaining) * '0':
                if num_children_packets:
                    if num_children_packets > 1:
                        print('\tRecursing from packet count with read limit on remaining!')
                        Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1)
                else:
                    print('\tRecursing from packet count on remaining!')
                    Create_Tree_From_Packets(remaining, parent_node, depth = depth + 1)
            else:
                print('\tNo remaining or all zeros!')
            

            

In [57]:
in_h = 'D2FE28'
in_h = '38006F45291200'
in_h = 'EE00D40C823060'

in_h = '8A004A801A8002F478'
#in_h = '620080001611562C8802118E34'
#in_h = 'C0015000016115A2E0802F182340'
#in_h = 'A0016C880162017C3686B18A3D4780'
in_h = data[0]


#in_h = 'C200B40A82'
#in_h = '04005AC33890'
#in_h = '880086C3E88112'
#in_h = 'CE00C43D881120'
#in_h = 'D8005AC2A8F0'
#in_h = 'F600BC2D8F'
#in_h = '9C005AC2F8F0'
#in_h = '9C0141080250320F1802104A08'

in_b_raw = bin(int(in_h, 16))[2:]
print('Raw (Size: {}):'.format(len(in_b_raw)), in_b_raw)

packet = add_leading_zeros_to_make_multiple_of_four(in_b_raw)
print('Padded (Size: {}):'.format(len(packet)), packet)


root = Packet_Node()
Create_Tree_From_Packets(packet, root, None, 0)

print()
root.Print_Tree()

Raw (Size: 5478): 1000100000110101001011100000000100100100011111111001101111101111011100110110100110000111110010001111110001110110011011011101100011000000000100101001111100000100101000000000010010111110011101101010001000110011100010011110110000000000001011001100001111010010000000010011010100100111001101010100010101001110000000001101010010000100000000011101100011110111000101111010001110110000000000000010000100010000110011100010110001000010110110011001111010000111100110000000001001010010110111101111101000110101100000000010101100111001110111010011011111001110011101110100110110010010010111100111110110011110101001101011000000110100011000000000101011001010100100000101000000000100110101010011110110101111110001110111000000011000100111101100010101111011000101110011001100000000010110100101001111010110111010101111111100000000001001011001001110100100111100100101011100111100001000000000100110001111110000000001000110011101000001101001110100000010100111011010110111000001000011000000001000001001001001

Create_Tree_From_Packets: 1001000010100010001100 , Read Limit: None , Recursion Depth: 35
	Packet: 100 100 0010100010001100
	Version: 100 = 4
	Type ID: 100 = 4 (Literal)

	Literal Packet!

	Literal (Size in bits: 4): 0101 = 5
	Encoding Bit Count: 5
	100 100 0010100010001100
	VVV TTT LLLLL
	Remaining bits: 00010001100
	Recursing from literal on remaining!

Create_Tree_From_Packets: 00010001100 , Read Limit: None , Recursion Depth: 36
	Packet: 000 100 01100
	Version: 000 = 0
	Type ID: 100 = 4 (Literal)

	Literal Packet!

	Literal (Size in bits: 4): 1100 = 12
	Encoding Bit Count: 5
	000 100 01100
	VVV TTT LLLLL
	Remaining bits: 
	No remaining or all zeros!
	Remaining bits: 111000000000000010000101010001000111100001101101000101101110001001111001100000000010101111000000000011100110010010010100011100011011111001100001011101100110000010100110011110100000000000100100000000011110011111001111010110000110011101000000000101001001111011100110001010010010101010110001110011110111001100010100100101010

	Expected number of sub-packets: 2 Actual Number of sub-packets 2
my total_length [16, 80] 96
	Remaining bits: 001010000000000011010000110011111111001111011100110101110100010000100010101111001000111110101111000011001110100010110000100000000010100001001001110001101010100101100110110110011011010111111001010101100110100110010111100001000000000100010111111001011110010101110100101110011000110010001101110010101111101010011000110001011100100000100100111100111100111111100110111100101100001101011001100000000010101101100000000010110100101111011111111100010000100110010111101111111110001000011111001101111111111010000111000110000000001010010100000000001111101011001111010111000000101001010011101111011010011100100000001110110011100110101010000101101011100000000010000100010110001001110111010111111010110000001110000
	Recursing from packet count with read limit on remaining!

Create_Tree_From_Packets: 0010100000000000110100001100111111110011110111001101011101000100001000101011110010001111101011110000110

In [58]:
def versions(root, version):
    
    for x in root.children:
        version += [x.version]
        versions(x, version)
            
version = []
versions(root, version)
sum(version)

977

In [59]:
def Compute_From_Tree(root):
    
    print('\nCompute_From_Tree', [x.type_id for x in root.children])
        
    if root.type_id == 4:
        print('base case', root.literal)
        return root.literal
    
    if root.type_id == 0:
        print('sum')
        res = [Compute_From_Tree(x) for x in root.children]
        print('returning', res)
        return sum(res)
        
    if root.type_id == 1:
        print('prod')
        res = 1
        for x in root.children:
            res *= Compute_From_Tree(x)
        return res

    if root.type_id == 2:
        print('min')
        return min([Compute_From_Tree(x) for x in root.children])
        
    if root.type_id == 3:
        print('max')
        return max([Compute_From_Tree(x) for x in root.children])
    
    if root.type_id == 5:
        print('ge')
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] > res[1])
    
    if root.type_id == 6:
        print('le')
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] < res[1])
    
    if root.type_id == 7:
        print('ee')
        res = [Compute_From_Tree(x) for x in root.children]
        print('res', res)
        return int(res[0] == res[1])

Compute_From_Tree(root.children[0])


Compute_From_Tree [3, 1, 3, 1, 4, 1, 4, 4, 1, 1, 1, 1, 1, 0, 3, 0, 4, 1, 4, 1, 3, 1, 1, 0, 2, 1, 4, 2, 4, 1, 4, 1, 1, 1, 1, 4, 0, 0, 1, 1, 0, 0, 1, 2, 1, 1, 2, 4, 1, 2, 1, 1, 3]
sum

Compute_From_Tree [4, 4, 4, 4]
max

Compute_From_Tree []
base case 15

Compute_From_Tree []
base case 199071281

Compute_From_Tree []
base case 499

Compute_From_Tree []
base case 190

Compute_From_Tree [4, 5]
prod

Compute_From_Tree []
base case 240

Compute_From_Tree [4, 4]
ge

Compute_From_Tree []
base case 3410

Compute_From_Tree []
base case 201

Compute_From_Tree [4]
max

Compute_From_Tree []
base case 7

Compute_From_Tree [4, 7]
prod

Compute_From_Tree []
base case 51786

Compute_From_Tree [0, 0]
ee

Compute_From_Tree [4, 4, 4]
sum

Compute_From_Tree []
base case 15

Compute_From_Tree []
base case 11

Compute_From_Tree []
base case 14
returning [15, 11, 14]

Compute_From_Tree [4, 4, 4]
sum

Compute_From_Tree []
base case 6

Compute_From_Tree []
base case 11

Compute_From_Tree []
base case 5
returni

101501020883